In [ ]:
import os
from PIL import Image
import argparse
import cv2
import torch
import numpy as np
import glob
from torch.cuda import amp
from tqdm import tqdm
import sys
from controlnet_aux.processor import Processor as controlet_processor
from anime_segmentation.train import AnimeSegmentation, net_names

In [ ]:
def get_mask(model, input_img, use_amp=True, s=640):
    input_img = (input_img / 255).astype(np.float32)
    h, w = h0, w0 = input_img.shape[:-1]
    h, w = (s, int(s * w / h)) if h > w else (int(s * h / w), s)
    ph, pw = s - h, s - w
    img_input = np.zeros([s, s, 3], dtype=np.float32)
    img_input[ph // 2 : ph // 2 + h, pw // 2 : pw // 2 + w] = cv2.resize(
        input_img, (w, h)
    )
    img_input = np.transpose(img_input, (2, 0, 1))
    img_input = img_input[np.newaxis, :]
    tmpImg = torch.from_numpy(img_input).type(torch.FloatTensor).to(model.device)
    with torch.no_grad():
        if use_amp:
            with amp.autocast():
                pred = model(tmpImg)
            pred = pred.to(dtype=torch.float32)
        else:
            pred = model(tmpImg)
        pred = pred.cpu().numpy()[0]
        pred = np.transpose(pred, (1, 2, 0))
        pred = pred[ph // 2 : ph // 2 + h, pw // 2 : pw // 2 + w]
        pred = cv2.resize(pred, (w0, h0))[:, :, np.newaxis]
        return pred

model = AnimeSegmentation.try_load(
    "isnet_is",
    "/home/aihao/workspace/DeepLearningContent/models/anime-seg/isnetis.ckpt",
    "cuda",
    img_size=1024,
)
model.eval()
model.to("cuda")

In [ ]:
for validation_path in os.listdir("../validation_images"):
    validation_path = os.path.join("../validation_images", validation_path)
    for image_path in os.listdir(validation_path):
        if "origin" not in image_path:
            continue
        img = cv2.cvtColor(
            cv2.imread(os.path.join(validation_path, image_path), cv2.IMREAD_COLOR),
            cv2.COLOR_BGR2RGB,
        )
        mask = get_mask(
            model, img, use_amp=False, s=min(1024, min(img.shape[0], img.shape[1]))
        )
        img = np.concatenate((mask * img + 1 - mask, mask * 255), axis=2).astype(
            np.uint8
        )
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGRA)
        if not os.path.exists(
            os.path.join(validation_path, image_path.replace("origin", "capture"))
        ):
            cv2.imwrite(
                os.path.join(validation_path, image_path.replace("origin", "capture")),
                img,
            )

In [ ]:
for validation_path in os.listdir("../validation_images"):
    validation_path = os.path.join("../validation_images", validation_path)
    for image_path in os.listdir(validation_path):
        if "2_capture" not in image_path:
            continue
        img = cv2.cvtColor(
            cv2.imread(os.path.join(validation_path, image_path), cv2.IMREAD_COLOR),
            cv2.COLOR_BGR2GRAY,
        )
        img = cv2.adaptiveThreshold(
            img,
            255,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            blockSize=5,
            C=7,
        )
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        if not os.path.exists(
            os.path.join(validation_path, image_path.replace("capture", "line"))
        ):
            cv2.imwrite(
                os.path.join(validation_path, image_path.replace("capture", "line")),
                img,
            )

In [ ]:
processor_type="scribble_hed"
process = controlet_processor(processor_type)
for validation_path in os.listdir("../validation_images"):
    validation_path = os.path.join("../validation_images", validation_path)
    for image_path in os.listdir(validation_path):
        if "2_capture" not in image_path:
            continue
        img = Image.open(os.path.join(validation_path, image_path))
        img = process(img, to_pil=True)
        img.save(os.path.join(validation_path,image_path.replace('capture',processor_type)))

In [ ]:
padding_image_path = "/home/aihao/workspace/StableDiffusionReferenceOnly/validation_images/4/2_capture.jpg"
img = Image.open(padding_image_path).convert("RGBA")
side_length = max(img.size[0], img.size[1])
filled_image = Image.new("RGBA", (side_length, side_length), (0, 0, 0, 0))
x_offset = (side_length - img.size[0]) // 2
y_offset = (side_length - img.size[1]) // 2
filled_image.paste(img, (x_offset, y_offset), img)
filled_image.save(padding_image_path)

In [ ]:
filled_image